In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.sequence import pad_sequences
import gensim.downloader as api
import ast

2023-06-22 12:56:25.433832: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-22 12:56:25.462002: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-22 12:56:25.580772: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-22 12:56:25.581961: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-22 12:56:26.363350: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
file_path = '~/code/TechLah/RevuSum/data/processed_kswdf.csv'
hotelreviews = pd.read_csv(file_path)
df = hotelreviews.copy()

In [3]:
df = df.groupby('Label', group_keys=False).apply(lambda x: x.sample(10000))
df['Label'].value_counts()

Label
0    10000
1    10000
Name: count, dtype: int64

In [4]:
X = df['Review']
X = [ast.literal_eval(string) for string in X]
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X,y, shuffle=True,test_size=0.3,random_state=1)

## Below doesnt work because data too big

In [5]:
# Train Word2Vec model on training data
# word2vec_model = Word2Vec(sentences=X_train, vector_size=100)

## Tried using pre-trained model instead

In [6]:
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [7]:
word2vec_model = api.load("glove-wiki-gigaword-50")

In [8]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

In [9]:
# Embed the training and test sentences
X_train_embed = embedding(word2vec_model, X_train)
X_test_embed = embedding(word2vec_model, X_test)


In [10]:
# Pad the training and test embedded sentences
X_train_emb_pad = pad_sequences(X_train_embed, dtype='float16', padding='post', maxlen=100)
X_test_emb_pad = pad_sequences(X_test_embed, dtype='float16', padding='post', maxlen=100)

In [12]:
X_train_emb_pad.shape

(14000, 100, 50)

In [11]:
# Pipeline Hashing vectorizer + Naive Bayes
pipeline_naive_bayes = make_pipeline(
    MultinomialNB()
)

# Define the grid of parameters
parameters = {
    'multinomialnb__alpha': [0.1, 1]
}

# Perform Grid Search
grid_search = GridSearchCV(
    pipeline_naive_bayes,
    parameters,
    scoring = "accuracy",
    cv = 5,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_emb_pad,y_train)

# Best score
print(f"Best Score = {grid_search.best_score_}")

# Best params
print(f"Best params = {grid_search.best_params_}")

Fitting 5 folds for each of 2 candidates, totalling 10 fits


ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/home/jack/.pyenv/versions/3.10.6/envs/RevuSum/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/jack/.pyenv/versions/3.10.6/envs/RevuSum/lib/python3.10/site-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/home/jack/.pyenv/versions/3.10.6/envs/RevuSum/lib/python3.10/site-packages/sklearn/naive_bayes.py", line 749, in fit
    X, y = self._check_X_y(X, y)
  File "/home/jack/.pyenv/versions/3.10.6/envs/RevuSum/lib/python3.10/site-packages/sklearn/naive_bay

In [ ]:
# Access the best parameters
best_params = grid_search.best_params_

# Use the best parameters to create a pipeline
pipeline_best = make_pipeline(
    MultinomialNB(alpha=best_params['multinomialnb__alpha'])
)

cv_nb_w2v = cross_validate(
    pipeline_best,
    X_train,
    y_train,
    scoring = "accuracy",
    cv=10
)

# Print the mean recall score
print(f"Mean Accuracy Score: {round(cv_nb_w2v['test_score'].mean(),2)}")

Mean Accuracy Score: 0.93


In [ ]:
w2v_nb_model = pipeline_best.fit(X_train,y_train)
y_pred = w2v_nb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9299354000575227
